<a href="https://colab.research.google.com/github/Karincheong/Call-For-Help-American-Sign-Language-Recognition-Using-Deep-Learning-Model/blob/main/LTSM_model_result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
#from keras.preprocessing import image
from keras.utils import load_img, img_to_array
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.models import load_model
import pickle

In [ ]:
with open('final_features.pickle', 'rb') as f:
    X= pickle.load(f)

with open('final_labels.pickle', 'rb') as l:
    y= pickle.load(l)

In [ ]:
# Shuffle the dataset
indices = np.arange(len(X))
np.random.shuffle(indices)
X = [X[i] for i in indices]
y = [y[i] for i in indices]

In [ ]:
# Convert the labels to integers and one-hot encode them
le = LabelEncoder()
y = le.fit_transform(y)
y = np_utils.to_categorical(y)

In [ ]:
# Split the dataset into training and testing sets
X_train, X_rem, y_train, y_rem = train_test_split(X, y, train_size=0.7, random_state=42)

In [ ]:
X_val, X_test, y_val, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

In [ ]:
# Reshape the features into 3D tensors for input to LSTM
X_train3 = np.expand_dims(X_train, axis=2)
X_val3 = np.expand_dims(X_val, axis=2)
X_test3 = np.expand_dims(X_test, axis=2)

In [ ]:
X_train1 = np.array(X_train).reshape(len(X_train), 1, -1)
X_val1 = np.array(X_val).reshape(len(X_val), 1, -1)
X_test1 = np.array(X_test).reshape(len(X_test), 1, -1)

In [ ]:
# Reshape the features into 3D tensors for input to LSTM
X_train2 = np.reshape(X_train, (len(X_train), 1, -1))
X_val2 = np.reshape(X_val, (len(X_val), 1, -1))
X_test2 = np.reshape(X_test, (len(X_test), 1, -1))

In [ ]:
#n_timesteps, n_features = 1, X_train[0].shape[0]

In [ ]:
X_train2.shape[1:]

(1, 131072)

In [ ]:
n_timesteps, n_features = X_train2.shape[1:]

In [ ]:

n_classes = len(y_train[0])
classifier = Sequential()
classifier.add(LSTM(256, input_shape=(n_timesteps, n_features)))
classifier.add(Dropout(0.5))
classifier.add(Dense(n_classes, activation='softmax'))

In [ ]:
# Compile the model with appropriate loss and metrics
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
classifier.fit(X_train2, y_train, epochs=10, batch_size=32, validation_data=(X_val2, y_val))

Epoch 1/10
153/153 [==============================] - 293s 2s/step - loss: 2.2681 - accuracy: 0.1524 - val_loss: 2.0996 - val_accuracy: 0.2242
Epoch 2/10
153/153 [==============================] - 280s 2s/step - loss: 2.2152 - accuracy: 0.1597 - val_loss: 2.1044 - val_accuracy: 0.2405
Epoch 3/10
153/153 [==============================] - 280s 2s/step - loss: 2.1748 - accuracy: 0.1745 - val_loss: 2.0914 - val_accuracy: 0.2109
Epoch 4/10
153/153 [==============================] - 287s 2s/step - loss: 2.1250 - accuracy: 0.1951 - val_loss: 2.0427 - val_accuracy: 0.3302
Epoch 5/10
153/153 [==============================] - 285s 2s/step - loss: 2.1250 - accuracy: 0.1964 - val_loss: 2.0405 - val_accuracy: 0.2586
Epoch 6/10
153/153 [==============================] - 271s 2s/step - loss: 2.1026 - accuracy: 0.2019 - val_loss: 2.0083 - val_accuracy: 0.3197
Epoch 7/10
153/153 [==============================] - 291s 2s/step - loss: 2.0432 - accuracy: 0.2252 - val_loss: 1.9801 - val_accuracy: 0.3387

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
filepath="LTSM_weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
classifier.fit(X_train2, y_train, epochs=100, batch_size=32, callbacks=callbacks_list, validation_data=(X_val2, y_val))

Epoch 1/100
153/153 [==============================] - ETA: 0s - loss: 1.9982 - accuracy: 0.2424
Epoch 1: val_accuracy improved from -inf to 0.34924, saving model to LTSM_weights-improvement-01-0.35.hdf5
153/153 [==============================] - 283s 2s/step - loss: 1.9982 - accuracy: 0.2424 - val_loss: 1.8948 - val_accuracy: 0.3492
Epoch 2/100
153/153 [==============================] - ETA: 0s - loss: 1.9801 - accuracy: 0.2526
Epoch 2: val_accuracy improved from 0.34924 to 0.40363, saving model to LTSM_weights-improvement-02-0.40.hdf5
153/153 [==============================] - 268s 2s/step - loss: 1.9801 - accuracy: 0.2526 - val_loss: 1.8871 - val_accuracy: 0.4036
Epoch 3/100
153/153 [==============================] - ETA: 0s - loss: 1.9342 - accuracy: 0.2718
Epoch 3: val_accuracy did not improve from 0.40363
153/153 [==============================] - 252s 2s/step - loss: 1.9342 - accuracy: 0.2718 - val_loss: 1.8575 - val_accuracy: 0.3931
Epoch 4/100
153/153 [========================

153/153 [==============================] - ETA: 0s - loss: 1.6209 - accuracy: 0.4056
Epoch 28: val_accuracy did not improve from 0.57729
153/153 [==============================] - 237s 2s/step - loss: 1.6209 - accuracy: 0.4056 - val_loss: 1.4364 - val_accuracy: 0.5573
Epoch 29/100
153/153 [==============================] - ETA: 0s - loss: 1.6310 - accuracy: 0.4015
Epoch 29: val_accuracy improved from 0.57729 to 0.60401, saving model to LTSM_weights-improvement-29-0.60.hdf5
153/153 [==============================] - 259s 2s/step - loss: 1.6310 - accuracy: 0.4015 - val_loss: 1.3849 - val_accuracy: 0.6040
Epoch 30/100
153/153 [==============================] - ETA: 0s - loss: 1.6194 - accuracy: 0.4220
Epoch 30: val_accuracy did not improve from 0.60401
153/153 [==============================] - 244s 2s/step - loss: 1.6194 - accuracy: 0.4220 - val_loss: 1.3559 - val_accuracy: 0.6021
Epoch 31/100
153/153 [==============================] - ETA: 0s - loss: 1.5889 - accuracy: 0.4271
Epoch 31: 

Epoch 56/100
153/153 [==============================] - ETA: 0s - loss: 1.3805 - accuracy: 0.4921
Epoch 56: val_accuracy improved from 0.67653 to 0.68034, saving model to LTSM_weights-improvement-56-0.68.hdf5
153/153 [==============================] - 297s 2s/step - loss: 1.3805 - accuracy: 0.4921 - val_loss: 1.1481 - val_accuracy: 0.6803
Epoch 57/100
153/153 [==============================] - ETA: 0s - loss: 1.3863 - accuracy: 0.5005
Epoch 57: val_accuracy did not improve from 0.68034
153/153 [==============================] - 296s 2s/step - loss: 1.3863 - accuracy: 0.5005 - val_loss: 1.1108 - val_accuracy: 0.6651
Epoch 58/100
153/153 [==============================] - ETA: 0s - loss: 1.3875 - accuracy: 0.4874
Epoch 58: val_accuracy did not improve from 0.68034
153/153 [==============================] - 298s 2s/step - loss: 1.3875 - accuracy: 0.4874 - val_loss: 1.1635 - val_accuracy: 0.6546
Epoch 59/100
153/153 [==============================] - ETA: 0s - loss: 1.4165 - accuracy: 0.47

Epoch 84/100
153/153 [==============================] - ETA: 0s - loss: 1.2357 - accuracy: 0.5519
Epoch 84: val_accuracy did not improve from 0.74237
153/153 [==============================] - 264s 2s/step - loss: 1.2357 - accuracy: 0.5519 - val_loss: 0.9313 - val_accuracy: 0.7071
Epoch 85/100
153/153 [==============================] - ETA: 0s - loss: 1.2931 - accuracy: 0.5251
Epoch 85: val_accuracy did not improve from 0.74237
153/153 [==============================] - 244s 2s/step - loss: 1.2931 - accuracy: 0.5251 - val_loss: 1.0023 - val_accuracy: 0.6947
Epoch 86/100
153/153 [==============================] - ETA: 0s - loss: 1.4027 - accuracy: 0.4927
Epoch 86: val_accuracy did not improve from 0.74237
153/153 [==============================] - 244s 2s/step - loss: 1.4027 - accuracy: 0.4927 - val_loss: 1.0152 - val_accuracy: 0.6966
Epoch 87/100
153/153 [==============================] - ETA: 0s - loss: 1.2932 - accuracy: 0.5304
Epoch 87: val_accuracy improved from 0.74237 to 0.75382,

In [ ]:
model_path = 'LTSM100.h5'
classifier.save(model_path)

In [ ]:
loss, accuracy = classifier.evaluate(X_test2, y_test)

33/33 [==============================] - 5s 131ms/step - loss: 0.9490 - accuracy: 0.7347
